<a href="https://colab.research.google.com/github/KaustubhK94/Automatic_Numberplate_Detection_Using_yolov8/blob/main/Content_grouping_based_on_text_information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install pdfminer.six

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Import necessary libraries
from bs4 import BeautifulSoup
import re
from langchain.document_loaders import PDFMinerPDFasHTMLLoader

# Load the PDF file using PDFMinerPDFasHTMLLoader
pdf_path = input("Enter the path of the PDF file: ")
loader = PDFMinerPDFasHTMLLoader(pdf_path)
data = loader.load()[0]   # entire PDF is loaded as a single Document

# Function to extract font information from a span
def extract_font_info(span):
    style = span.get('style')
    if not style:
        return None

    # Extract font-related styles from the CSS style.
    font_family = re.findall(r'font-family:(.*?);', style)
    font_size = re.findall(r'font-size:(.*?);', style)

    return {'font-family': font_family[0].strip(), 'font-size': font_size[0].strip()} if font_family and font_size else None

# Assuming content is a single string containing HTML content of all pages
soup = BeautifulSoup(data.page_content, 'html.parser')

# Find all span elements in the HTML
spans = soup.find_all('span')

# Initialize variables to keep track of the current font style, the current text being accumulated, and a list to store snippets.
cur_font_info = None
cur_text = ''
snippets = []   # Collect all snippets with font information

# Get user input for the page number
page_number = int(input("Enter the page number: "))

# Validate the page number
if 0 <= page_number < len(data.page_content):
    # Loop through all spans
    for sp in spans:
        # Check if the current element has the same font as the previous one. If yes, accumulate the text.
        font_info = extract_font_info(sp)
        if font_info == cur_font_info:
            cur_text += sp.get_text()
        else:
            # If not, store the current snippet and reset variables for the new font style.
            snippets.append((cur_text, cur_font_info))
            cur_font_info = font_info
            cur_text = sp.get_text()

    # Add the last accumulated snippet to the list
    snippets.append((cur_text, cur_font_info))

    # Now, 'snippets' contains grouped text content with font information across all pages
    semantic_snippets = snippets

    # Get user input for the output file path
    output_file_path = input("Enter the output file path (including the file name with .txt extension): ")

    # Write the snippets to the output file
    with open(output_file_path, 'w') as output_file:
        for snippet, font_info in semantic_snippets:
            if font_info:
                output_file.write(f'<style font-family="{font_info["font-family"]}" font-size="{font_info["font-size"]}">\n')
            output_file.write(f'\t{snippet}\n')
            if font_info:
                output_file.write('</style>\n\n' + "=" * 50 + "\n\n")

    print(f"Data has been written to {output_file_path}")
else:
    print(f"Page {page_number} is out of range.")
